# Exercise 1

What is the total frequency search range for L2 and L5 carrier, if the TCXO has the following specifications:

TCXO offset within +-3ppm

In [ ]:
tcxo_offset = 3e-6
f_L1 = 1575.42e6
f_L2 = 1227.60e6
f_L5 = 1176.45e6
f_L1_offset = f_L1 * tcxo_offset
f_L2_offset = f_L2 * tcxo_offset
f_L5_offset = f_L5 * tcxo_offset
print(f"L5 Doppler search range: {-f_L5_offset} Hz to {f_L5_offset} Hz")
print(f"L2 Doppler search range: {-f_L2_offset} Hz to {f_L2_offset} Hz")
# print(f"L1 Doppler search range: {-f_L1_offset} Hz to {f_L1_offset} Hz")